In [9]:
!git clone https://github.com/amccaugh/polarizer

# Installation from https://colab.research.google.com/drive/1KoAQ1C_BNtGV3sVvZCnNZaER9rstmy0s#scrollTo=RapkYvTXL7Cd
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines box2d box2d-kengz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  autotools-dev cmake-data file ibverbs-providers libarchive13 libfabric1
  libhwloc-dev libhwloc-plugins libhwloc5 libibverbs-dev libibverbs1
  libjsoncpp1 libltdl-dev libltdl7 liblzo2-2 libmagic-mgc libmagic1
  libnl-3-200 libnl-route-3-200 libnuma-dev libnuma1 libopenmpi2 libpciaccess0
  libpsm-infinipath1 librdmacm1 librhash0 libtool libuv1 ocl-icd-libopencl1
  openmpi-bin openmpi-common swig3.0
Suggested packages:
  cmake-doc ninja-build lrzip libhwloc-contrib-plugins libtool-doc openmpi-doc
  pciutils autoconf automaken gcj-jdk swig-doc swig-examples swig3.0-examples
  swig3.0-doc
The following NEW packages will be installed:
  autotools-dev cmake cmake-data file ibverbs-providers libarchive13
  libfabric1 libhwloc-dev libhwloc-pl

In [0]:
import os
import sys
module_path = os.path.abspath(os.path.join('/content/polarizer/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [0]:

# #%%
# %load_ext autoreload
# %autoreload 2
import gym_polarizer # Register the environment 'polarizer-v0'
import numpy as np
from matplotlib import pyplot as plt
plt.ion()

# if __name__ == '__main__':
#     print('hello')
#     # import pdb; pdb.set_trace()
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

env = gym.make('polarizer-v0')
# env = gym.make('CartPole-v1')
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run

# Defined in https://github.com/hill-a/stable-baselines/blob/master/stable_baselines/ppo2/ppo2.py
    # Proximal Policy Optimization algorithm (GPU version).
    # Paper: https://arxiv.org/abs/1707.06347
    # :param policy: (ActorCriticPolicy or str) The policy model to use (MlpPolicy, CnnPolicy, CnnLstmPolicy, ...)
    # :param env: (Gym environment or str) The environment to learn from (if registered in Gym, can be str)
    # :param gamma: (float) Discount factor
    # :param n_steps: (int) The number of steps to run for each environment per update
    #     (i.e. batch size is n_steps * n_env where n_env is number of environment copies running in parallel)
    # :param ent_coef: (float) Entropy coefficient for the loss caculation
    # :param learning_rate: (float or callable) The learning rate, it can be a function
    # :param vf_coef: (float) Value function coefficient for the loss calculation
    # :param max_grad_norm: (float) The maximum value for the gradient clipping
    # :param lam: (float) Factor for trade-off of bias vs variance for Generalized Advantage Estimator
    # :param nminibatches: (int) Number of training minibatches per update. For recurrent policies,
    #     the number of environments run in parallel should be a multiple of nminibatches.
    # :param noptepochs: (int) Number of epoch when optimizing the surrogate
    # :param cliprange: (float or callable) Clipping parameter, it can be a function
    # :param verbose: (int) the verbosity level: 0 none, 1 training information, 2 tensorflow debug
    # :param tensorboard_log: (str) the log location for tensorboard (if None, no logging)
    # :param _init_setup_model: (bool) Whether or not to build the network at the creation of the instance

def learning_rate_anneal(proportion):
    """ The learning rate can be a function accepting a proportion-of-steps-
    completed value from 1 (time=0) to 0 (time=end), so this one performs annealing """
    start = 1e-2
    stop = 1e-5
    learning_rate = np.exp((np.log(stop)-np.log(start))*(1-proportion) + np.log(start))
    return learning_rate

model = PPO2(MlpPolicy, env, verbose=1,
                gamma=0.99, n_steps=1024, ent_coef=0.01, learning_rate=learning_rate_anneal, vf_coef=0.5,
                max_grad_norm=0.5, lam=0.95, nminibatches=8, noptepochs=8, cliprange=0.2,
                tensorboard_log='/tmp/tensorboard/', _init_setup_model=True)
#                gamma=0.99, n_steps=128, ent_coef=0.01, learning_rate=1e-3, vf_coef=0.5,
#                max_grad_norm=0.5, lam=0.95, nminibatches=4, noptepochs=4, cliprange=0.2,
#                tensorboard_log=None, _init_setup_model=True)

import tensorflow as tf
import timeit

# # See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# with tf.device('/cpu:0'):
model.learn(total_timesteps=3000000)

#%%
all_obs = []
all_rewards = []
all_dones = []

obs = env.reset()
for i in range(10000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    all_obs.append(obs[0])
    all_rewards.append(rewards[0])
    all_dones.append(dones[0])
    env.render()

all_obs = np.array(all_obs)
all_rewards = np.array(all_rewards)
fig, axs = plt.subplots(3,1, sharex = True)
axs[0].plot(all_rewards)
axs[1].plot(all_obs)
axs[2].plot(all_dones)
plt.show()


------------------------------------
| approxkl           | 0.026577193 |
| clipfrac           | 0.35742188  |
| ep_rewmean         | nan         |
| eplenmean          | nan         |
| explained_variance | 0.0689      |
| fps                | 639         |
| nupdates           | 1           |
| policy_entropy     | 1.9202999   |
| policy_loss        | 0.021480087 |
| serial_timesteps   | 1024        |
| time_elapsed       | 3.81e-06    |
| total_timesteps    | 1024        |
| value_loss         | 21.289925   |
------------------------------------
------------------------------------
| approxkl           | 0.027497197 |
| clipfrac           | 0.35131836  |
| ep_rewmean         | nan         |
| eplenmean          | nan         |
| explained_variance | 0.00186     |
| fps                | 885         |
| nupdates           | 2           |
| policy_entropy     | 1.9129101   |
| policy_loss        | 0.010192908 |
| serial_timesteps   | 2048        |
| time_elapsed       | 1.6         |
|